In [243]:
# test: Query a form from fulcrum (Preventive Maintenance) and publish to postgresql
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb
from datetime import datetime, timedelta
# import credentials
from config.secrets import *
from tdutils.pgrestutil import Postgrest
import json

In [204]:
pd.options.display.max_columns = 2000

In [205]:
form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
key = FULCRUM_CRED.get("api_key")


In [252]:
pgrest = Postgrest("http://transportation-data-test.austintexas.io/signal_pms", auth=JOB_DB_API_TOKEN_test)

In [222]:
"""Summary

Attributes:
    form_id (str): Description
    key (TYPE): Description
    pgrest (TYPE): Description
"""
# test: Query a form from fulcrum (Preventive Maintenance) and publish to postgresql
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb
from datetime import datetime, timedelta
# import credentials
from config.secrets import *

from tdutils.pgrestutil import Postgrest


form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
key = FULCRUM_CRED.get("api_key")

# create postgrest instance
pgrest = Postgrest("http://transportation-data-test.austintexas.io/signal_pms", auth=JOB_DB_API_TOKEN_test)


# test: Query a form from fulcrum (Preventive Maintenance)

def recur_dict(col_names, elements):
    """Summary
    
    Args:
        col_names (TYPE): Description
        elements (TYPE): Description
    
    Returns:
        TYPE: Description
    """


    # print(type(elements))

    for element in elements:

        if type(element) == dict:
            col_names[element.get("key")] = element.get("data_name")

            for key, value in element.items():
                if type(value) == list:
                    recur_dict(col_names, value)

    return col_names



def get_col_names(form_id):
    """Summary
    get a dictionary with {"key": "label"} pair for all columns in a data
    base
    
    
    
    Args:
        form_id (TYPE): Description
    
    
    
    Returns:
        TYPE: Description
    
    
    
    """

    form = fulcrum.forms.find(form_id)
    elements = form.get("form").get("elements")

    col_names = {}

    col_names = recur_dict(col_names, elements)

    return col_names

def get_records(form_id):
    """Summary
    
    Args:
        form_id (TYPE): Description
    
    Returns:
        TYPE: Description
    """
    # initiate a dataframe

    records_dirty = fulcrum.records.search(url_params = {"form_id":form_id})

    records = pd.DataFrame()



    for record in records_dirty["records"]:
        form_values = record["form_values"]
        for key, value in form_values.items():
            # print(value)
            if type(value) == dict and "choice_values" in value:
                value = value["choice_values"]
                if type(value) == list and len(value) == 2:
                    if key == "fce3":
                        form_values[key] = value[1]
                
            if type(value) == list and len(value) == 1:
                form_values[key] = value[0]
            
                
        form_values["_server_updated_at"] = record["created_at"]
        form_values["_record_id"] = record["id"]
        new_row = pd.DataFrame([form_values], columns=form_values.keys())
    #     print(new_row)
        records = pd.concat([new_row, records], axis =0, sort=False).reset_index(drop=True)

    return records

def interpret_col_name(records):
    """Summary
    
    Args:
        records (TYPE): Description
    
    Returns:
        TYPE: Description
    """
    return records.rename(columns=col_names)


def clean_pm(records):
    """Summary
    
    Args:
        records (TYPE): Description
    
    Returns:
        TYPE: Description
    """

    
    df = records.copy()
    
#     print(df)

    df["signal_id"] = df["signal"].str.split("|").str[0]

    # rename record_id to fulcrum ID
    df = df.rename(columns={'_record_id': 'fulcrum_id'})

    df["_server_updated_at"] = df["_server_updated_at"].str.strip("CDT")
    df["pm_completed_datetime"] = pd.to_datetime(df["_server_updated_at"], format = "%Y-%m-%d %H:%M:%S")
    df["pm_completed_date"] = df["pm_completed_datetime"].dt.date
    
#     print(df["technicians"])

    df["pm_completed_by"] = df["technicians"]#.str.split(",").str[1]
    df["modified_current"] = datetime.now().replace(microsecond=0).isoformat(" ")

    df["modified_current"] =pd.to_datetime(df["modified_current"], format = "%Y-%m-%d %H:%M:%S")

    df["modified_date"] = df["modified_current"] + timedelta(minutes = 20)

    cleaned_record = df[["signal_id", "fulcrum_id", "pm_completed_date", "modified_date", "pm_completed_by"]]

    return cleaned_record

def get_pgrest_records():
    # the datetime converstin for modified_date is not right. The time part are missing

    results = pgrest.select('')
    results = pd.DataFrame(results)
    results["modified_date"] = pd.to_datetime(results["modified_date"], format='%Y-%m-%dT%H:%M:%S')
#     results["modified_date"] = results["modified_date"](format="'%Y-%m-%dT%H:%M:%S'")
    results["pm_completed_date"] = pd.to_datetime(results["pm_completed_date"])
    
    return results

def get_most_recent():
    """Summary

    
    Returns:
        TYPE: Description
    """
    results = pgrest.select('fulcrum_id=eq.abc123')

    return results


def main():
    """Summary
    """
    
    
    
    pass



In [228]:
    fulcrum = fc.Fulcrum(key = key)
    forms = fulcrum.forms.search(url_params={'id': form_id})
    col_names = get_col_names(form_id)
    records = get_records(form_id)
    
    
    

    records = interpret_col_name(records)
    
    

    # pdb.set_trace()

    fulcrum_records = clean_pm(records)
    
    print(records)
#     print(forms)

   load_switches                       emergency_vehicle_preemption  \
0             OK  {'other_values': ['could not test '], 'choice_...   
1             OK                                                NaN   
2             OK                                                NaN   
3             OK                                                NaN   
4             OK                                                NaN   
5             OK                                                NaN   
6             OK                                                NaN   
7             OK                                                NaN   
8             OK                                                NaN   
9             OK                                                NaN   
10            OK                                                NaN   
11            OK                                                NaN   
12            OK                                                NaN   
13    

In [230]:
pgrest_records

,fulcrum_id,modified_date,pm_completed_by,pm_completed_date,signal_id
0,abc123,2018-09-20,john.clary@austintexas.gov,2018-09-20,9999


In [118]:
pgrest_records["modified_date"]

0    2018-09-20T00:00:00
Name: modified_date, dtype: object

In [93]:
pgrest_records.head()

,fulcrum_id,modified_date,pm_completed_by,pm_completed_date,signal_id
0,abc123,2018-09-20,john.clary@austintexas.gov,2018-09-20,9999


In [232]:
fulcrum_records.head()

,signal_id,fulcrum_id,pm_completed_date,modified_date,pm_completed_by
0,289,ff4687c8-b044-4f7b-aabb-b0e267a214be,2018-09-21,2018-10-15 15:28:41,toby.turner@austintexas.gov
1,238,119530f5-e0ce-4a3d-9d5f-39a78c39f71f,2018-09-06,2018-10-15 15:28:41,toby.turner@austintexas.gov
2,229,c5b17f94-6e61-4c7a-84b6-de51ac49774c,2018-09-06,2018-10-15 15:28:41,toby.turner@austintexas.gov
3,440,acb8724f-f499-43eb-81f5-da3566539418,2018-09-04,2018-10-15 15:28:41,toby.turner@austintexas.gov
4,675,c2251729-2927-4556-b558-4cc2178a976f,2018-09-04,2018-10-15 15:28:41,toby.turner@austintexas.gov


In [233]:
pgrest_records

,fulcrum_id,modified_date,pm_completed_by,pm_completed_date,signal_id
0,abc123,2018-09-20,john.clary@austintexas.gov,2018-09-20,9999


In [234]:
def prepare_payload(fulcrum_records, pgrest_records):
    # compare the modified date and fulcrum id in fulcrum records and 
    # in postgrest record. 
    
    
    payload = fulcrum_records[~fulcrum_records['fulcrum_id'].isin(pgrest_records['fulcrum_id'])]
    
    return payload


In [235]:
payload = prepare_payload(fulcrum_records, pgrest_records)

In [236]:
payload

,signal_id,fulcrum_id,pm_completed_date,modified_date,pm_completed_by
0,289,ff4687c8-b044-4f7b-aabb-b0e267a214be,2018-09-21,2018-10-15 15:28:41,toby.turner@austintexas.gov
1,238,119530f5-e0ce-4a3d-9d5f-39a78c39f71f,2018-09-06,2018-10-15 15:28:41,toby.turner@austintexas.gov
2,229,c5b17f94-6e61-4c7a-84b6-de51ac49774c,2018-09-06,2018-10-15 15:28:41,toby.turner@austintexas.gov
3,440,acb8724f-f499-43eb-81f5-da3566539418,2018-09-04,2018-10-15 15:28:41,toby.turner@austintexas.gov
4,675,c2251729-2927-4556-b558-4cc2178a976f,2018-09-04,2018-10-15 15:28:41,toby.turner@austintexas.gov
5,673,47793b3e-4885-4003-8674-fbab0795ff2e,2018-08-30,2018-10-15 15:28:41,toby.turner@austintexas.gov
6,674,a71673ab-e49f-4889-9675-bec0bf5eb03d,2018-08-30,2018-10-15 15:28:41,toby.turner@austintexas.gov
7,172,9de056f4-c6cb-4787-b5e3-8f82a7015d3a,2018-08-28,2018-10-15 15:28:41,toby.turner@austintexas.gov
8,4050,c5fbb66b-1a79-46f2-9ec9-276631639d4d,2018-08-29,2018-10-15 15:28:41,toby.turner@austintexas.gov
9,914,0717226b-0578-4d6c-8d4b-8c8760d96543,2018-08-29,2018-10-15 15:28:41,toby.turner@austintexas.gov


In [253]:
def upload_pgrest(payload):
    
    payload = payload.to_dict(orient='index')
    payload = json.dumps(payload, indent=4, sort_keys=True, default=str)

    
    upload_result = pgrest.upsert(payload)
    
    return upload_result
    

In [254]:
upload_pgrest(payload)

HTTPError: 400 Client Error: Bad Request for url: http://transportation-data-test.austintexas.io/signal_pms

In [195]:
records.sample(10)

,4de9,21c6,6080,ffc1,33cc,3f80,a460,146e,7840,93dd,ecf2,6e0a,e1b4,ea0f,fce3,f36f,20de,f95d,8354,5a23,552f,6e42,8594,609a,55d6,4a48,c01c,3a14,eb53,8968,_server_updated_at,_record_id,8926,f283,52d5,8435,0d1f
8,OK,NaN,4050 | QUINLAN PARK RD / CANYON GLEN DR,OK,GOOD,no,no,OK,No Preemption,OK,OK,APS,no,yes,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': 'c409d7ba-c5e6-4af1-96cd-2ed82c42...,No Camera,yes,No APS Beepers,OK,yes,No UPS,OK,OK,13:29,13:17,Cleaned,Replaced,2018-08-29T18:21:54Z,c5fbb66b-1a79-46f2-9ec9-276631639d4d,NaN,NaN,NaN,NaN,NaN
39,OK,NaN,1017 | ANDERSON MILL RD / CASHELL WOOD DR,OK,EXCELLENT,no,NaN,OK,No Preemption,OK,OK,APS,yes,no,"{'other_values': [], 'choice_values': ['thomas...",No Obstruction,OK,{'photo_id': '4376342c-5f9a-4312-87c3-d1f1be4a...,No Camera,yes,OK,OK,yes,No UPS,"{'other_values': ['d4'], 'choice_values': []}",OK,08:15,07:03,Cleaned,Replaced,2018-08-07T14:15:16Z,a91eae80-1723-41bf-9828-e51ef8525648,NaN,"[{'updated_at': '1533643768', 'created_locatio...",NaN,{'photo_id': '0f7fdb70-4153-4925-b36a-32accccc...,NaN
2,OK,NaN,229 | CESAR CHAVEZ ST / SANDRA MURAIDA WAY,OK,GOOD,no,no,OK,No Preemption,OK,OK,Bulldog,yes,no,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '8ba73151-85b8-4433-b183-f5ddfaa8...,OK,yes,No APS Beepers,OK,yes,No UPS,OK,OK,09:23,04:40,Cleaned,Replaced,2018-09-06T14:23:23Z,c5b17f94-6e61-4c7a-84b6-de51ac49774c,Cleaned,"[{'updated_at': '1536230172', 'created_locatio...",NaN,NaN,NaN
14,OK,NaN,716 | ANDERSON LN SVRD / NORTHCREST BLVD,OK,POOR,yes,no,OK,No Preemption,OK,OK,Bulldog,yes,yes,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '03ad4216-0577-4d18-a7c1-1ef326e4...,{'other_values': ['wired wrong placed cabinet ...,yes,No APS Beepers,OK,yes,No UPS,OK,OK,06:46,05:02,Cleaned,OK,2018-08-28T11:44:03Z,c83b6e2d-cce2-426b-9846-89b58aaa6c8c,"{'other_values': ['fried'], 'choice_values': []}","[{'updated_at': '1535456009', 'created_locatio...",1. All loops cut road base good \n2. Fiber ris...,{'photo_id': '882f6a8d-f7dc-4af1-b671-e4f911bb...,NaN
16,OK,NaN,639 | LAMAR BLVD SVRD / RESEARCH BLVD SVRD,OK,GOOD,yes,yes,OK,No Preemption,OK,OK,Bulldog,yes,yes,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '1a87b687-92cc-4c9c-a06b-150d9fda...,No Camera,yes,No APS Beepers,OK,yes,{'other_values': ['Ups is dead needs batteries...,OK,OK,08:05,07:17,Cleaned,{'other_values': ['could not change needs fya'...,2018-08-24T15:29:55Z,c9d00d04-a0d5-4e3c-a86e-0c064551660b,NaN,"[{'updated_at': '1535118925', 'created_locatio...",1. South bound loops bad\n2.Ups is dead needs ...,{'photo_id': '8d9d2d6a-f289-4854-8dbc-1a932a74...,On most of the poles
21,OK,OK,609 | DUVAL RD / RESEARCH BLVD SVRD,OK,GOOD,no,no,OK,Emergency Vehicle,OK,OK,Bulldog,yes,no,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '0cb4a2e8-8ae3-448e-b985-c37c1954...,OK,yes,No APS Beepers,OK,yes,{'other_values': ['Ups is dead needs batteries...,OK,OK,07:30,05:18,Cleaned,Replaced,2018-08-21T12:18:24Z,b795123b-c1ad-41dc-a544-56dca24a3832,"{'other_values': ['ok'], 'choice_values': []}","[{'updated_at': '1534853705', 'created_locatio...",NaN,{'photo_id': '4fa2eeb0-3076-46e8-b815-d3887ee6...,NaN
18,OK,NaN,808 | BRAKER LN / RESEARCH BLVD SVRD,OK,GOOD,yes,no,OK,No Preemption,OK,OK,Bulldog,yes,no,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '1f75306a-bdbf-476d-8d3b-47460e92...,OK,yes,No APS Beepers,OK,yes,{'other_values': ['Ups is dead needs batteries...,OK,OK,06:29,05:00,Cleaned,{'other_values': ['ok and programmed 172.16.1...,2018-08-23T12:07:32Z,6614833d-c98a-4972-a908-668407bfb6e2,Cleaned,"[{'updated_at': '1535021257', 'created_locatio...",1.South bound loop wire bad road way and pull ...,{'photo_id': 'f3e1b36a-1f35-48f0-b1aa-16e02506...,NaN
10,OK,NaN,948 | QUINLAN PARK RD / STEINER RANCH BLVD,Repaired,GOOD,yes,no,OK,No Preemption,OK,OK,Bulldog,no,yes,toby.turner@austintexas.gov,No Obstruction,OK,{'photo_id': '0aa86b9d-2b0a-4b64-9544-62128750...,No Camera,yes,No APS Beepers,OK,yes,{'other_value